In [86]:
import sys, math
import random as rn

Neste exemplo temos sempre "hardwired" $\,w = 256\,$, $\,|w|=8\,$, $\,n = 64\,$ e $\,m = 112\,$

In [87]:
N  = 64
M  = 48
W  = 8
w  = 256

l1 = math.ceil(M//W)
l2 = math.floor(math.log(l1*(w-1),2)//W) + 1
l  = l1 + l2

l,l1,l2

(8, 6, 2)

In [88]:
nbits = lambda     : rn.getrandbits(N) 
wbits = lambda     : rn.getrandbits(8)

def mbits():
    m = rn.getrandbits(M)
    return list(m.to_bytes(l1,byteorder='big'))

def checksum(m):
    C = sum([w-1-m[j] for j in range(l1)])
    return list(C.to_bytes(l2,byteorder='big'))

In [89]:
def wots(k,x,i=1):
    if i <= 0:
        return x
    if i == 1:
        return hash(k^x)
    return wots(wots(k,x,i-1),x)


In [98]:
# lei de grupo
i = rn.randrange(w)
j = rn.randrange(w)
x = nbits()
k = nbits()

wots(k,x,i+j) == wots(wots(k,x,i),x,j)


True

## WOTS

In [91]:
def KG():
    x  = nbits()
    sk = [nbits() for j in range(l)]
    pk = [wots(sk[j],x,w-1) for j in range(l)]
    return (sk,pk,x)
    
def Sig(m,sk,x):
    m_ = m + checksum(m)
    return [wots(sk[j],x,m_[j]) for j in range(l)]

def Vrfy(sig,m,pk,x):
    m_ = m + checksum(m)
    return all([wots(sig[j],x , w - 1 - m_[j]) == pk[j] for j in range(l)])
    

In [92]:
m  = mbits()

sk,pk,x = KG()

sig = Sig(m,sk,x)

Vrfy(sig,m,pk,x)


True

## WOTS+

In [93]:
## Público e global

def c(r,x,i=1):
    assert i <= len(r)
    if i <= 0:
        return x
    return hash(r[i-1] ^ c(r,x,i-1))

In [94]:
def KG_():
    r    = [nbits()  for i in range(w-1)]
    sk   = [nbits()        for j in range(l)]
    pk   = [c(r,sk[j],w-1) for j in range(l)]
    return (sk,pk,r)

def Sig_(mess,sk,r):
    m = mess + checksum(mess)
    return [c(r,sk[j],m[j]) for j in range(l)]

def Vrfy_(sig,mess,pk,r):
    m = mess + checksum(mess)
    return all([c(r[m[j]:], sig[j], w-1-m[j]) == pk[j]  for j in range(l)])

In [95]:
m  = mbits()

sk,pk,ref = KG_()

In [96]:
sig = Sig_(m,sk,ref)

Vrfy_(sig,m,pk, ref)

True

In [97]:
## Propriedade de grupo
l = w
r = [nbits()  for i in range(l-1)]

def teste(i,j):
    x = nbits()
    return c(r,x,i+j) == c(r[i:],c(r,x,i),j)

## teste

while True:
    i = rn.randrange(1,l)
    j = rn.randrange(1,l)
    try:
        print(f"índices {i} e {j} verificam o teste? ",teste(i,j))
    except:
        print(f"indices {i} e {j} fora do domínio")
        break
        

índices 111 e 111 verificam o teste?  True
índices 110 e 52 verificam o teste?  True
índices 34 e 216 verificam o teste?  True
índices 19 e 176 verificam o teste?  True
indices 109 e 158 fora do domínio
